In [49]:
import folium
import requests
import pandas as pd
from folium import plugins

In [58]:
CLIENT_ID = 'your CLIENT_ID'
CLIENT_SECRET = 'your CLIENT_SECRET'

In [42]:
request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20201212',
    "section": "restaurant",
    "near": "Coimbatore",
    "radius": 2000,
    "limit": 100}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [43]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'totalResults', 'suggestedBounds', 'groups'])

In [34]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Coimbatore', 'Coimbatore')

In [35]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 11.02175643384067, 'lng': 76.98551288708627},
  'sw': {'lat': 10.988383350870064, 'lng': 76.94793716821773}},
 94)

In [36]:
d["geocode"]

{'what': '',
 'where': 'coimbatore',
 'center': {'lat': 11.00555, 'lng': 76.96612},
 'displayString': 'Coimbatore, Tamil Nādu, India',
 'cc': 'IN',
 'geometry': {'bounds': {'ne': {'lat': 11.300586493514743,
    'lng': 77.25951292698001},
   'sw': {'lat': 10.818872163666398, 'lng': 76.72485979883756}}},
 'slug': 'coimbatore-india',
 'longId': '72057594039201801'}

In [37]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [38]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [44]:
items = d["groups"][0]["items"]
print("Number of items: %i" % len(items))
items[0]

Number of items: 94


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4ede31ec0aaf49e02e010f39',
  'name': 'The Cinemas',
  'location': {'address': 'Brookefields Plaza, 67-71, Krishnasamy Rd',
   'crossStreet': 'Krishnasamy Road',
   'lat': 11.009058599302957,
   'lng': 76.95939969301797,
   'labeledLatLngs': [{'label': 'display',
     'lat': 11.009058599302957,
     'lng': 76.95939969301797}],
   'postalCode': '641001',
   'cc': 'IN',
   'city': 'Coimbatore',
   'state': 'Tamil Nadu',
   'country': 'India',
   'formattedAddress': ['Brookefields Plaza, 67-71, Krishnasamy Rd (Krishnasamy Road)',
    'Coimbatore 641001',
    'Tamil Nadu',
    'India']},
  'categories': [{'id': '4bf58dd8d48988d180941735',
    'name': 'Multiplex',
    'pluralName': 'Multiplexes',
    'shortName': 'Cineplex',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/movietheater_',
     'suffi

In [46]:
df_raw = []
cnt = 0
for item in items:
    cnt = cnt+1
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    if not "address" in location:
        continue
    address = location["address"]
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("Found %i Restaurants !" % len(df))
df.head()

Found 37 Restaurants !


,uid,name,shortname,address,postalcode,lat,lng
0,4ede31ec0aaf49e02e010f39,The Cinemas,Cineplex,"Brookefields Plaza, 67-71, Krishnasamy Rd",641001,11.009059,76.959400
1,4cb5dad71b0af04d534bcc25,Brookefields Mall,Mall,"67-71, Krishnasamy Road",641001,11.008875,76.959430
2,4dc514ba2271f27051233f65,Junior Kuppana,Indian,Sarojini Street,641009,11.011269,76.965129
3,52c1abf4498edd7c505a59f9,Barista,Café,Brookefields,641001,11.009090,76.958880
4,4eb0d70677c89bf55f31b6b4,"Vivanta by Taj - Surya, Coimbatore",Hotel,Race Course Road,641018,11.002026,76.973712


In [47]:
cbe_center = d["geocode"]["center"]
cbe_center

{'lat': 11.00555, 'lng': 76.96612}

In [53]:
map_cbe = folium.Map(location=[cbe_center["lat"], cbe_center["lng"]], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_cbe)

add_markers(df)
hm_data = df[["lat", "lng"]].values.tolist()
map_cbe.add_child(plugins.HeatMap(hm_data))

map_cbe

## Results:
Based on these results, one possibly good location for new Restaurant would be in **Dr.Nanjappa Road** near **Jawaharlal Nehru Stadium**

In [57]:
lat = 11.00555
lng = 76.96612

folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="Our Restaurant!",
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_cbe)
map_cbe